# Подготовка данных для обучения модели детекции СИЗ

Этот ноутбук содержит все шаги для подготовки данных:
1. Извлечение кадров из видео
2. Автоматическая предразметка
3. Разделение на train/val
4. Проверка данных


In [ ]:
# Импорт необходимых библиотек
import os
import sys
from pathlib import Path
import cv2
from ultralytics import YOLO
import shutil
import random

# Добавляем корень проекта в путь
project_root = Path('.').resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("✅ Библиотеки загружены")
print(f"Корень проекта: {project_root}")


## Шаг 1: Извлечение кадров из видео

Поместите все видео в папку `videos/` и запустите этот блок.


In [ ]:
def extract_frames_from_video(video_path, output_dir, step=30):
    \"\"\"
    Извлекает кадры из видео с заданным шагом.
    \"\"\"
    cap = cv2.VideoCapture(str(video_path))
    
    if not cap.isOpened():
        print(f"⚠️  Не удалось открыть видео: {video_path}")
        return 0
    
    frame_count = 0
    saved_count = 0
    video_name = video_path.stem
    
    print(f"  Обработка: {video_path.name}")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % step == 0:
            frame_filename = output_dir / f"{video_name}_frame_{saved_count:06d}.jpg"
            cv2.imwrite(str(frame_filename), frame)
            saved_count += 1
        
        frame_count += 1
    
    cap.release()
    print(f"  ✓ Сохранено кадров: {saved_count} (из {frame_count} всего)")
    return saved_count

def auto_extract_frames(videos_dir="videos", output_dir="data/images/train", step=30):
    videos_path = Path(videos_dir)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    if not videos_path.exists():
        print(f"❌ Папка '{videos_dir}' не найдена!")
        print(f"Создайте папку '{videos_dir}' и поместите туда видео файлы.")
        return
    
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv', '.m4v']
    video_files = []
    for ext in video_extensions:
        video_files.extend(videos_path.glob(f"*{ext}"))
        video_files.extend(videos_path.glob(f"*{ext.upper()}"))
    
    if len(video_files) == 0:
        print(f"❌ Не найдено видео файлов в '{videos_dir}'!")
        print(f"Поддерживаемые форматы: {', '.join(video_extensions)}")
        return
    
    print(f"Найдено видео: {len(video_files)}")
    total_saved = 0
    
    for i, video_file in enumerate(video_files, 1):
        print(f"\n[{i}/{len(video_files)}] {video_file.name}")
        saved = extract_frames_from_video(video_file, output_path, step)
        total_saved += saved
    
    print(f"\n✅ Извлечено {total_saved} кадров в {output_dir}/")
    return total_saved

# Запуск извлечения кадров
print("=== ИЗВЛЕЧЕНИЕ КАДРОВ ИЗ ВИДЕО ===")
extracted = auto_extract_frames()


## Шаг 2: Автоматическая предразметка

Используем предобученную YOLOv8 для создания предварительных bounding boxes вокруг людей.


In [ ]:
def auto_prelabel(images_dir="data/images/train", labels_dir="data/labels/train", conf_threshold=0.3):
    \"\"\"
    Автоматическая предразметка с использованием предобученной модели.
    \"\"\"
    images_path = Path(images_dir)
    labels_path = Path(labels_dir)
    labels_path.mkdir(parents=True, exist_ok=True)
    
    image_extensions = ['*.jpg', '*.jpeg', '*.png']
    image_files = []
    for ext in image_extensions:
        image_files.extend(images_path.glob(ext))
        image_files.extend(images_path.glob(ext.upper()))
    
    if len(image_files) == 0:
        print(f"❌ Не найдено изображений в {images_dir}")
        return
    
    print(f"Найдено изображений: {len(image_files)}")
    
    # Загружаем модель
    print("Загрузка предобученной модели YOLOv8n...")
    model = YOLO("yolov8n.pt")
    
    # COCO класс 'person' = 0
    person_class = 0
    total_annotations = 0
    processed = 0
    
    for i, image_file in enumerate(image_files, 1):
        image = cv2.imread(str(image_file))
        if image is None:
            continue
        
        results = model(image, conf=conf_threshold, verbose=False)
        boxes = results[0].boxes
        
        if boxes is None or len(boxes) == 0:
            processed += 1
            continue
        
        # Фильтруем людей
        person_boxes = []
        for box in boxes:
            cls = int(box.cls[0].cpu().numpy())
            if cls == person_class:
                person_boxes.append(box)
        
        if len(person_boxes) == 0:
            processed += 1
            continue
        
        # Создаем разметку
        label_file = labels_path / (image_file.stem + ".txt")
        with open(label_file, 'w') as f:
            img_h, img_w = image.shape[:2]
            for box in person_boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                
                center_x = (x1 + x2) / 2 / img_w
                center_y = (y1 + y2) / 2 / img_h
                width = (x2 - x1) / img_w
                height = (y2 - y1) / img_h
                
                # Класс 0 для всех (потом в LabelImg поменяете на 1 для жилетов)
                f.write(f"0 {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
        
        total_annotations += len(person_boxes)
        processed += 1
        if i % 10 == 0:
            print(f"Обработано: {i}/{len(image_files)}")
    
    print(f"\n✅ Предразметка завершена!")
    print(f"Обработано: {processed}/{len(image_files)} изображений")
    print(f"Создано аннотаций: {total_annotations}")
    print(f"Разметка сохранена в: {labels_dir}")

# Запуск предразметки
print("=== АВТОМАТИЧЕСКАЯ ПРЕДРАЗМЕТКА ===")
auto_prelabel()


In [ ]:
def split_dataset(train_images_dir, train_labels_dir, val_images_dir, val_labels_dir, val_ratio=0.2, seed=42):
    \"\"\"
    Разделяет датасет на train/val.
    \"\"\"
    random.seed(seed)
    
    # Создаем папки val
    Path(val_images_dir).mkdir(parents=True, exist_ok=True)
    Path(val_labels_dir).mkdir(parents=True, exist_ok=True)
    
    # Получаем изображения
    train_images_path = Path(train_images_dir)
    images = list(train_images_path.glob("*.jpg")) + list(train_images_path.glob("*.png"))
    
    if len(images) == 0:
        print(f"❌ Не найдено изображений в {train_images_dir}")
        return
    
    random.shuffle(images)
    val_count = int(len(images) * val_ratio)
    val_images = images[:val_count]
    
    print(f"Всего изображений: {len(images)}")
    print(f"Train: {len(images) - val_count} ({(1-val_ratio)*100:.1f}%)")
    print(f"Val: {val_count} ({val_ratio*100:.1f}%)")
    
    moved_count = 0
    for img in val_images:
        # Перемещаем изображение
        dest_img = Path(val_images_dir) / img.name
        shutil.move(str(img), str(dest_img))
        
        # Перемещаем разметку
        label_file = Path(train_labels_dir) / (img.stem + ".txt")
        if label_file.exists():
            dest_label = Path(val_labels_dir) / label_file.name
            shutil.move(str(label_file), str(dest_label))
            moved_count += 1
        else:
            print(f"⚠️  Нет разметки для {img.name}")
    
    print(f"\n✅ Перемещено {moved_count} пар в val/")
    return moved_count

# Разделение датасета
print("=== РАЗДЕЛЕНИЕ НА TRAIN/VAL ===")
train_images = "data/images/train"
train_labels = "data/labels/train"
val_images = "data/images/val"
val_labels = "data/labels/val"

split_dataset(train_images, train_labels, val_images, val_labels)


## Шаг 4: Проверка данных

Проверяем структуру и корректность разметки перед обучением.


In [ ]:
def check_data_structure():
    \"\"\"
    Проверяет структуру данных и корректность разметки.
    \"\"\"
    print("=== ПРОВЕРКА СТРУКТУРЫ ДАННЫХ ===")
    
    # Проверяем папки
    required_dirs = [
        "data/images/train", "data/images/val",
        "data/labels/train", "data/labels/val"
    ]
    
    print("\n📁 Проверка папок:")
    for dir_path in required_dirs:
        if Path(dir_path).exists():
            img_count = len(list(Path(dir_path).glob("*.jpg"))) + len(list(Path(dir_path).glob("*.png")))
            txt_count = len(list(Path(dir_path).glob("*.txt")))
            print(f"✅ {dir_path}: {img_count} изображений, {txt_count} разметок")
        else:
            print(f"❌ Отсутствует: {dir_path}")
    
    # Проверяем соответствие изображений и разметки
    print("\n🔍 Проверка соответствия:")
    train_missing_labels = 0
    train_missing_images = 0
    
    for img in Path("data/images/train").glob("*.jpg"):
        label = Path("data/labels/train") / (img.stem + ".txt")
        if not label.exists():
            train_missing_labels += 1
    
    for label in Path("data/labels/train").glob("*.txt"):
        img = Path("data/images/train") / (label.stem + ".jpg")
        if not img.exists():
            train_missing_images += 1
    
    if train_missing_labels > 0:
        print(f"⚠️  {train_missing_labels} изображений без разметки в train")
    if train_missing_images > 0:
        print(f"⚠️  {train_missing_images} разметок без изображений в train")
    
    if train_missing_labels == 0 and train_missing_images == 0:
        print("✅ Все файлы соответствуют друг другу")
    
    # Проверяем формат разметки
    print("\n📋 Проверка формата разметки:")
    invalid_labels = 0
    for label_file in Path("data/labels/train").glob("*.txt"):
        try:
            with open(label_file) as f:
                for line_num, line in enumerate(f, 1):
                    parts = line.strip().split()
                    if len(parts) != 5:
                        print(f"❌ {label_file.name}:{line_num} - неверное количество параметров")
                        invalid_labels += 1
                        continue
                    
                    class_id = int(parts[0])
                    if class_id not in [0, 1]:
                        print(f"⚠️  {label_file.name}:{line_num} - неверный класс {class_id}")
                        invalid_labels += 1
                        continue
                    
                    coords = [float(x) for x in parts[1:]]
                    if any(x < 0 or x > 1 for x in coords):
                        print(f"⚠️  {label_file.name}:{line_num} - координаты вне диапазона")
                        invalid_labels += 1
                        continue
        except Exception as e:
            print(f"❌ Ошибка в {label_file.name}: {e}")
            invalid_labels += 1
    
    if invalid_labels == 0:
        print("✅ Формат разметки корректен")
    else:
        print(f"⚠️  Найдено {invalid_labels} проблем в разметке")
    
    # Общая статистика
    train_imgs = len(list(Path("data/images/train").glob("*.jpg")))
    val_imgs = len(list(Path("data/images/val").glob("*.jpg")))
    total_imgs = train_imgs + val_imgs
    
    print(f"\n📊 СТАТИСТИКА:")
    print(f"Всего изображений: {total_imgs}")
    print(f"Train: {train_imgs} ({train_imgs/total_imgs*100:.1f}%)")
    print(f"Val: {val_imgs} ({val_imgs/total_imgs*100:.1f}%)")
    
    if total_imgs >= 100:
        print("✅ Достаточно данных для обучения")
    else:
        print("⚠️  Рекомендуется больше данных (минимум 100 изображений)")
    
    return total_imgs > 0

# Запуск проверки
data_ok = check_data_structure()
